In [1]:
import gymnasium as gym
import os
import sys
sys.path.append("./jsbgym")
import jsbgym
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack

In [2]:
env = gym.make("A320-AltitudeHoldTask-Shaping.STANDARD-NoFG-v0", render_mode="human")
# env = gym.make("A320-TurnHeadingControlTask-Shaping.STANDARD-NoFG-v0", render_mode="human")

In [3]:
env.reset()



     JSBSim Flight Dynamics Model v1.2.3 Jun  7 2025 19:28:34
            [JSBSim-ML v2.0]

JSBSim startup beginning ...



(array([ 5.00000000e+03, -8.32667268e-17,  1.05706940e-18,  4.21950000e+02,
        -2.61884750e-14,  2.84217094e-14,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00, -3.72529030e-09]),
 {})

In [4]:
for episode in range(1, 6):
    obs, info = env.reset()
    done = False
    total_reward = 0
    while not done:
        obs, reward, done, _, info = env.step(env.action_space.sample())
        total_reward += reward
    print("Total Reward for episode {} is {}".format(episode, total_reward))

Total Reward for episode 1 is 30.47424847000618
Total Reward for episode 2 is 30.52953425764168
Total Reward for episode 3 is 30.45134133590662
Total Reward for episode 4 is 30.419178833569685
Total Reward for episode 5 is 30.443158638285162


In [4]:
class TrainAndLoggingCallback(BaseCallback):
    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(
                self.save_path, "best_model_{}".format(self.n_calls)
            )
            self.model.save(model_path)

        return True

In [5]:
CHECKPOINT_DIR = "./train/engine_alt_hold/"
LOG_DIR = "./logs/engine_alt_hold/"

In [6]:
callback = TrainAndLoggingCallback(check_freq=100000, save_path=CHECKPOINT_DIR)

In [ ]:
model = PPO("MlpPolicy", env, tensorboard_log=LOG_DIR, device="cpu")

/home/pietbcm/diff_thrust_rl/venv/lib/python3.12/site-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


In [8]:
model.learn(total_timesteps=1000000, callback=callback)
model.save("JSBSim_1000000_steps")

KeyboardInterrupt: 

## Visualization

In [13]:
env = gym.make("A320-AltitudeHoldTask-Shaping.STANDARD-FG-v0", render_mode="human_fg")
env.reset()
model = PPO.load(os.path.join("train/engine_alt_hold", "best_model_200000"), device="cuda")



     JSBSim Flight Dynamics Model v1.2.3 Jun  7 2025 19:28:34
            [JSBSim-ML v2.0]

JSBSim startup beginning ...



/home/pietbcm/diff_thrust_rl/venv/lib/python3.12/site-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


In [14]:
env.render()

In [15]:
for episode in range(1, 6):
    obs, _ = env.reset()
    done = False
    total_reward = 0
    while not done:
        action, _ = model.predict(obs)
        obs, reward, done, _, info = env.step(action)
        total_reward += reward
    print("Total Reward for episode {} is {}".format(episode, total_reward))

/home/pietbcm/diff_thrust_rl/jsbgym/jsbgym/environment.py:131: UserWarning: If training, use NoFG instead of FG in the env_id. Using FG will cause errors while training after a while.
  warnings.warn(


Total Reward for episode 1 is 30.41560335915624
Total Reward for episode 2 is 30.51048880596329


KeyboardInterrupt: 

In [17]:
env.close()